## Imports

In [3]:
import H5toDataframe as readH5
import apiconfiguration as api
import influxServices as influxServ

## API configuration

In [ ]:
write_api = api.get_write_api()
query_api = api.get_query_api()

## Read data from HDF5

In [ ]:
df = readH5.convertAllH5Files()

In [ ]:
# Query script
query = (f'from(bucket:"test5")\
|> range(start: 0)\
|> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")')

result = query_api.query(org=api.org, query=query)

print(len(result))

In [ ]:
result_dfq = query_api.query_data_frame(org=api.org, query=query)
print(result_dfq)

In [5]:
# Write data to InfluxDB
influxServ.upload_dict_with_frames(df, write_api)

NameError: name 'df' is not defined